In [1]:
# Import GBApy
#import gba

import sys
import gba

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Create a non full column rank model

To create a non full-column-rank toy model, use the script `./scripts/generate_toy_model.py` (see Github project note).

E.g. here, I used equal kcat values:

```bash
python ./scripts/generate_toy_model.py -path ./toy_models -name efm2 -efms 2 -kcat 10 -gradient 0.0
```

# 2. Load the model + basic manipulations

## 2.1. Load the model

In [2]:
model = gba.read_csv_model("efm2", "../toy_models")
model.summary()

Category,Count
Nb metabolites,6
Nb external metabolites,2
Nb internal metabolites,4
Category,Count
Nb reactions,5
Nb exchange reactions,2
Nb internal reactions,2
Category,Count
Column rank,4
Is full column rank?,False


Note that the model is not full column rank (table "Matrix rank").

In [3]:
model.kcat_f

array([10.  , 10.  , 10.  , 10.  ,  4.55])

For this model, the $k_\text{cat}$ value of the reaction `rxn1` (10 h<sup>-1</sup>) is slightly higher than for `rxn2` (9.99 h<sup>-1</sup>). We can expect both EFMs to have very close optimal growth rates.

## 2.2. Explore the fitness landscape of the model

We will now vary the flux of the reaction `rxn2` between 0.0 and 1.0.
As `rxn1` and `rxn2` are transporters, their `f` value will always vary between 0 and 1 thanks to the normalization of mass conservation.
By doing so, we can vizualise the fitness landscape of a flux depending on `rxn2`.

In the example below, for each new value of `rxn2`, the steps are:
- to clear the list of constant reactions,
- to set up the constant value of `rxn2`,
- to find an initial solution given this value,
- to set the initial point $f_0$,
- to run the gradient ascent.

The trick here to avoid very long solving times is to set up the maximal time of the gradient ascent to a small value (here, `max_time = 10`).

<strong>WARNING:</strong> For some reasons, the gradient ascent does not work for values close to 0.0 or 1.0. Here, `rxn2` ranges from 0.1 to 0.9.

In [35]:
res   = pd.DataFrame()
label = 1
model.clear_all_trajectories()
for rxn2 in np.arange(0.1, 1.0, 0.1):
    model.clear_constant_reactions()
    model.add_constant_reaction("rxn2", rxn2)
    model.solve_local_linear_problem()
    model.set_f0(model.LP_solution)
    model.gradient_ascent(label=label, condition_id="1", max_time=10, track=True, variables=["f", "v", "p", "b", "c"])
    # Get the last line of model.GA_tracker
    res = pd.concat([res, model.GA_tracker.iloc[-1:]])
    label += 1
res

,label,condition,iter,dt,t,mu,fixed,f_rxn1,f_rxn2,f_rxn3,...,p_rxn4,p_Ribosome,b_C1,b_C2,b_AA,b_Protein,c_C1,c_C2,c_AA,c_Protein
259,1,1,271,0.0,0.976001,0.611144,259,0.9,0.1,0.885321,...,3.164101,58.812525,0.014679,0.004893,0.060939,0.919489,4.990929,1.663643,20.719197,312.62623
513,2,1,254,0.008,1.024,0.60996,253,0.8,0.2,0.786177,...,5.708497,58.659825,0.013823,0.006911,0.060835,0.918431,4.69971,2.349855,20.683741,312.266694
763,3,1,250,0.008,0.992,0.609301,249,0.7,0.3,0.687079,...,8.139645,58.574965,0.012921,0.008459,0.060777,0.917844,4.393165,2.876002,20.664023,312.06681
1009,4,1,246,0.008,0.96,0.608952,245,0.6,0.4,0.588042,...,10.51341,58.530015,0.011958,0.009764,0.060746,0.917532,4.065803,3.319715,20.653574,311.960908
1254,5,1,255,0.0,0.912003,0.608841,244,0.5,0.5,0.489085,...,12.852464,58.515823,0.010915,0.010915,0.060736,0.917434,3.711128,3.711128,20.650275,311.927469
1500,6,1,246,0.008,0.96,0.608952,245,0.4,0.6,0.390236,...,15.169523,58.530015,0.009764,0.011958,0.060746,0.917532,3.319715,4.065803,20.653574,311.960908
1750,7,1,250,0.008,0.992,0.609301,249,0.3,0.7,0.291541,...,17.473648,58.574965,0.008459,0.012921,0.060777,0.917844,2.876002,4.393165,20.664023,312.06681
2004,8,1,255,0.0016,1.0176,0.60996,253,0.2,0.8,0.193089,...,19.773442,58.659825,0.006911,0.013823,0.060835,0.918431,2.349855,4.69971,20.683741,312.266694
2264,9,1,270,0.0,0.992013,0.611144,259,0.1,0.9,0.095107,...,22.081862,58.812525,0.004893,0.014679,0.060939,0.919489,1.663643,4.990929,20.719197,312.626231


We can vizualize a beautiful fitness landscape with two optima.

In [36]:
fig = model.create_figure("Fitness landscape of rxn3")
model.add_trajectory(fig, source="data", x_var="f_rxn3", y_var="mu", name="Fitness landscape of rxn3", data=res)
fig.show()